---
title: 位置情報したい
---

In [1]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
import hvplot.pandas

print(f"Pandas: {pd.__version__}")
print(f"GeoPandas: {gpd.__version__}")
print(f"HvPlot: {hvplot.__version__}")


Pandas: 2.2.2
GeoPandas: 0.14.3
HvPlot: 0.9.2


In [2]:
read_from = Path("./_static/phyphox/_GPS_2024-04-21_14-00-07/RawData.csv")
read_from.exists()


True

In [3]:
names = [
    "time",
    "latitude",
    "longitude",
    "altitude",
    "altitude_wgs84",
    "speed",
    "direction",
    "distance",
    "horizontal_accuracy",
    "vertical_accuracy",
    "satellites",
]

data = pd.read_csv(read_from, names=names, skiprows=1)
len(data)


4596

In [5]:
data.hvplot.scatter(x="time", y="latitude")
data.hvplot.scatter(x="time", y="longitude")
data.hvplot.scatter(x="longitude", y="latitude")


:Scatter   [longitude]   (latitude)

In [10]:
data.hvplot.explorer()


BokehModel(combine_events=True, render_bundle={'docs_json': {'f0b86a25-a5a8-44c5-afef-768fdab9712e': {'version…

In [8]:
gps = gpd.GeoDataFrame(data[["latitude", "longitude", "altitude"]])
